In [ ]:
'''
Here, we describe the code we use to simulate a population of binaries under the influence of a population of subhalos.
We describe the code in order of blocks in the evol.py code.
'''

# Overview:
The primary goal of this tutorial is to teach the reader how to run Monte Carlo simulations of binaries interacting with perturbing subhalos as described in the paper. Here, the code is contained in the evol.py file, which one must run using the terminal. In this notebook, we go through blocks of the code, explaining its purpose and how it works. We strongly advise readers to read Section III of the paper before attempting to use the code.

Before we begin, it is useful to give an overview of what we are trying to do. We want to set limits on subhalo populations using a catalog of wide binaries. As discussed in the paper, this requires us to predict the present-day orbital distribution of binaries $\phi(\vec{q})$ given an initial orbital distribution $\phi_{0}(\vec{q_{0}}$, where $\vec{q} = (a,e,\psi,M)$ denotes a binary orbital state. We found

$$
\begin{aligned}
    \phi(\vec{q}) = \int d\vec{q}_0 \ S(\vec{q}|\vec{q}_0) \ \phi_0(\vec{q}_0),
\end{aligned}
$$

where $S(\vec{q}|\vec{q}_0) \ \phi_0(\vec{q}_0)$ denotes the scattering matrix. It is the probability that, from the time the binary population was assembled to the present-day, the binary's orbital state evolves from $\vec{q}_{0}$ to $\vec{q}$ due to encounters with the perturber population. 

Looking at the equation above, we can simulate the evolution of a large number of binaries with orbital states sampled from $\phi_{0}(\vec{q}_{0})$ to calculate $\phi(\vec{q})$ directly. However, as discussed in Section III, the initial semimajor axis distribution is not well-understood, so we wish to marginalize over the possible distributions for our constraints to be independent of the semimajor axis distribution. It would be very inefficient to run simulations for each possible initial semimajor axis distribution, so we follow a technique introduced by Yoo et al. As we specify the distributions in $(e,\psi,M)$, let's integrate over these parameters to write the above equation as
$$
\begin{aligned}
    \phi(\vec{q}) = \int d\vec{a}_0 \ S(\vec{q}|a_0) \ \phi_0(a_{0}),
\end{aligned}
$$
where $S(\vec{q}|a_0)$ is now the scattering matrix integrated over $(e,\psi,M)$. This equation suggests that, if we can specify the scattering matrix over a range of initial semimajor axes $a_{0}$, then we need only convolve it with the initial semimajor axis distribution $\phi_{0}(a_{0})$ to calculate $\phi(\vec{q})$. Therefore, instead of performing a set of simulations for all the initial semimajor axis distributions, we can simulate binaries with semimajor axes $a_{0}$ to estimate $S(\vec{q}|a_0)$. To evolve any initial distribution of binaries, we simply perform the integral above. 

In practice, we estimate $S(\vec{q}|a_0)$ over bins of $a_{0}$.

$$  
\begin{aligned}
    S(\vec{q}|\vec{q}_0) = \int\prod_{i=0}^{N-1}\left [ \ d\vec{q}_i \ f_{i}(\vec{q}_{i+1} |\vec{q}_i)\right ],
\end{aligned}
$$

# I. Monte Carlo Simulations of Perturber-Binary Encounters (file: evol_pm.py)
## 1. Load Perturber-Binary System
The first step is to specify our population of binaries and perturbers. To leading-order, the perturbers and stellar binaries  uniformly distributed in space and both exhibit Maxwellian velocity profiles with velocity dispersion $v_{0}$. In this case, the population of perturbers are completely specified by their individual masses $M_{p}$ and the number density $n_{p} = f_{p} \times \frac{\rho}{M_{p}}$, where $f_{p}$ is the perturber fraction and $\rho$ is the local dark matter density $\rho_{DM} = 0.39 \ GeV/cm^{3}$. The stellar binaries are specified by the total mass $M$ and semimajor axis $a$. Since we are interested at how the perturbers act on the binaries, we are interested in the *relative* velocity profile. This is simply a Maxwellian distribution with velocity dispersion $\sqrt{2} v_{0}$.

An encounter between a perturber and a binary is specified by the impact parameter of the perturber relative to the center of the binary, their relative orientations, and the velocity of the perturber relative to the binary. As shown in Report III, this is specified by the parameters $(p,\theta,\phi,v)$. Therefore, generating an encounter from a perturber on a binary requires us to sample the random variable $(p,\theta,\phi,v;m,M,a,v_{0})$ from its associated PDF. 

In [3]:
# Initialization

## packages and functions
import timeit as timeit
import sys
import os
import numpy as np
import math as math 
from scipy.stats import maxwell
from scipy.integrate import quad, dblquad
from scipy.special import erf

ti = timeit.default_timer() ## start a timer

## input parameters (in the actual code, you set these values using the terminal)
a0 = float(0.1)    # initial binary semimajor axis (pc)    # perturber mass (M_solar)
str_a0 = str.format('{0:.9f}',a0)
log_m = float(3) # perturber mass (M_solar)
str_log_m = str.format('{0:.3f}',log_m)
m = 10**log_m
log_rs = float(-1) # NFW scale radius (pc)
str_log_rs = str.format('{0:.3f}',log_rs)
rs = 10**log_rs
log_c = float(0) # NFW concentration
str_log_c = str.format('{0:.3f}',log_c)
c = 10**log_c
log_f = float(0.2) # halo fraction [setting max number of time steps]
f = 10**log_f
str_log_f = str.format('{0:.3f}',log_f)

## conversion factors:
kg_g = 1E-3
Msolar_kg = 5.02785E-31 # Msolar/kg
pc_m = 3.24078E-17 # pc/m
Gyr_yr = pow(10,-9) # Gyr/yr
yr_s = 3.17098E-8 # yr/s
km_m = 1/1000 # km/m
kg_GeV = 1.78266E-27 # kg / (GeV / c**2)
m_cm = pow(10,-2) # m/cm
pc_AU = 4.84814e-6 # pc/AU
AU_pc = 206265  # AU/pc

## fixed parameters 
v0 = 240 # circular velocity (km/s) 
vesc = 533 # local escape velocity (km/s)
T = 10 # evolution time (Gyr)
rho_pdg = 0.39 # Local DM Density (GeV / cm3)
rho = 0.39 * (Msolar_kg) * (kg_GeV) * (pc_m)**(-3) * (m_cm)**(-3) #[Ed]

## constants
G = 4.30091e-3   # Grav Constant (pc (M_solar)^-1 (km/s)**2)

In [ ]:
## set up maximum impact parameter and time steps

### Compute the average inverse velocity for the truncated velocity profile

def v_theta_(cos_theta,v,vesc):
    return -v*cos_theta + np.sqrt(v*v*cos_theta*cos_theta + (vesc*vesc - v*v))

def fv_no_vesc_(v, sigma):
    return v*v*np.exp(-v*v / 4 / sigma / sigma)

def dfv_dcos_theta_eff_(v_theta,sigma):
    return (1/4) * (-2 * v_theta * np.exp( - v_theta * v_theta / sigma / sigma ) 
                 + np.sqrt(np.pi) * sigma * erf(v_theta / sigma) ) 

def inv_v_avg_(sigma,vesc,normalization):
    v_avg_eff = ( dblquad(lambda cos_theta, v: 
                                     fv_no_vesc_(v, sigma) * dfv_dcos_theta_eff_(v_theta_(cos_theta,v/2,vesc),sigma) / v, 0, 2*vesc, 
                                     lambda v: 0, lambda v: 1)[0] )
    return v_avg_eff / normalization

sigma = v0/math.sqrt(2)/math.sqrt(2)
normalization = ( dblquad(lambda cos_theta, v: 
                                     fv_no_vesc_(v, sigma) * dfv_dcos_theta_eff_(v_theta_(cos_theta,v/2,vesc),sigma), 0, 2*vesc, 
                                     lambda v: 0, lambda v: 1)[0] )
inv_v_avg = inv_v_avg_(sigma,vesc,normalization)

### max impact parameter when the fractional energy injection falls below energy_fraction
def p_max_(a,m,M,inv_v_avg,f,energy_fraction): 
    bar_C = 2*G*m * 2*G*m / 2 * inv_v_avg
    E = G * M / 2 / a
    inv_factor = np.pi * (f * rho * T / m) * bar_C / E
    bar_delta_Eps_val = inv_factor**(-1) * energy_fraction
    #if m >= 10:
    #    return a * np.sqrt(1 + 2 * bar_delta_Eps_val**(-1) * (1 + np.sqrt(1 + bar_delta_Eps_val))) # pc
    #else:
    #    return (m/10)**(1/2) * a * np.sqrt(1 + 2 * bar_delta_Eps_val**(-1) * (1 + np.sqrt(1 + bar_delta_Eps_val))) # pc
    return a * np.sqrt(1 + 2 * bar_delta_Eps_val**(-1) * (1 + np.sqrt(1 + bar_delta_Eps_val))) # pc


### load number of time steps needed to evolve for the specified evolution time
def delta_t_(m,inv_v_avg,f,R):    # length of time step in Gyr
    delta_t_val = m / (f * rho * np.pi * R**2) * inv_v_avg
    delta_t_val_Gyr = delta_t_val * ( (km_m) / (pc_m) ) * ( (Gyr_yr) * (yr_s) ) 
    return delta_t_val_Gyr  # Gyr

log_eps = float(-2)   # threshold fractional energy injection defining p_max
energy_fraction = 10**log_eps
str_log_eps = str.format('{0:.2f}',log_eps)
p_max = p_max_(1,m,1,inv_v_avg,1,energy_fraction) # p_max value (f=1 for efficiency, M =1 standard stable)
delta_t = delta_t_(m,inv_v_avg,f,p_max)    # delta_t value
N = int(T / delta_t)    # number of time steps